In [1]:
import numpy as np

In [176]:
np.set_printoptions(precision=16)
np.set_printoptions(suppress=True)

In [3]:
def stress_to_trac(n):
    nx, ny, nz = n
    mat = np.array([
        [nx, 0, 0, ny, nz, 0],
        [0, ny, 0, nx, 0, nz],
        [0, 0, nz, 0, nx, ny]
    ])
    return mat

def s2t(n1, n2):
    return np.vstack((stress_to_trac(n1), stress_to_trac(n2)))

def t2s(n1, n2):
    return np.linalg.pinv(s2t(n1, n2))

def t2t(n1, n2):
    s2t = np.vstack((stress_to_trac(n1), stress_to_trac(n2)))
    t2s = np.linalg.pinv(s2t)
    return s2t.dot(t2s)

In [132]:
M = sp.Matrix([
    [1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [nxp, 0, nyp, 0, 0],
    [0, nyp, nxp, 0, 0],
    [0, 0, 0, nxp, nyp]
])
Minv = M.pinv()

In [5]:
import sympy as sp
nx, ny, nz, nx2, ny2, nz2 = sp.symbols('nx, ny, nz, nx2, ny2, nz2', real = True)
nxp, nyp = sp.symbols('nxp, nyp', real = True)
tvar = sp.symbols('t', real = True)
M = sp.Matrix([
    [1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [nxp, 0, nyp, 0, 0],
    [0, nyp, nxp, 0, 0],
    [0, 0, 0, nxp, nyp]
])
Minv = M.pinv()
symbolic_t2t = M * Minv
symbolic_t2t.simplify()
symbolic_t2t = symbolic_t2t.subs(nxp, sp.cos(tvar)).subs(nyp, sp.sin(tvar))

In [9]:
for x in np.random.rand(100):
    t = 2 * np.pi * x
    symbolic = np.array(symbolic_t2t.subs(tvar, t)).astype(np.float64)
    n1 = [1,0,0]
    n2 = [np.cos(t), np.sin(t),0]
    numeric = t2t(n1, n2)
    np.testing.assert_almost_equal(symbolic, numeric)

In [154]:
def outer(a, b):
    return sp.Matrix([[a[i] * b[j] for j in range(3)] for i in range(3)])

def rotation_matrix(axis, theta):
    cross_mat = sp.Matrix([[0, -axis[2], axis[1]], [axis[2], 0, -axis[0]], [-axis[1], axis[0], 0]])
    outer_mat = outer(axis, axis)
    idmat = sp.eye(3)
    rot_mat = sp.cos(theta) * idmat + sp.sin(theta) * cross_mat + (1 - sp.cos(theta)) * outer_mat
    return rot_mat

def L(v):
    return sp.sqrt(sum([v[d] ** 2 for d in range(len(v))]))
    
def rotate_to_xaxis(v):
    to_target = sp.Array([1, 0, 0])
    axis = (v + to_target) / 2
    axis_mag = L(axis)
    if axis_mag == 0:
        axis = sp.Array([0,0,1])
    else:
        axis /= axis_mag
    theta = sp.pi
    rot_mat = rotation_matrix(axis, theta)
    return rot_mat

def rotate2_to_xyplane(v):
    xaxis = sp.Array([1, 0, 0])
    yaxis = sp.Array([0, 1, 0])
    ydot2 = v[1] / L(v[1:])
    theta = -sp.acos(ydot2) * sp.sign(v[2])
    rot_mat = rotation_matrix(xaxis, theta)
    return rot_mat

In [155]:
R1 = rotate_to_xaxis(sp.Array([nx, ny, nz]))
R2 = rotate2_to_xyplane(R1.dot([nx2, ny2, nz2]))
RR = R2 * R1

In [156]:
rot_n2 = RR.dot([nx2, ny2, nz2])
theta = sp.atan2(rot_n2[1], rot_n2[0])

In [157]:
symbolic_R6 = sp.Matrix([[0] * 6] * 6)
symbolic_R6[:3,:3] = RR
symbolic_R6[3:,3:] = RR

In [158]:
final_symbolic = symbolic_R6.T * symbolic_t2t.subs(tvar, theta) * symbolic_R6

In [180]:
from sympy.utilities.lambdify import lambdify
final_symbolic_fnc = lambdify([nx, ny, nz, nx2, ny2, nz2], final_symbolic[1,3])

In [160]:
def sym_t2t(n1, n2):
    return final_symbolic.subs([(nx, n1[0]), (ny, n1[1]), (nz, n1[2]), (nx2, n2[0]), (ny2, n2[1]), (nz2, n2[2])])

In [183]:
for i in range(5):
    n1 = np.random.rand(3)
    n1 /= np.linalg.norm(n1)
    n2 = np.random.rand(3)
    n2 /= np.linalg.norm(n2)
    numeric = t2t(n1, n2)
    symbolic = sym_t2t(n1, n2)
    np.testing.assert_almost_equal(symbolic, numeric)

In [185]:
import pickle
pickle.dump(final_symbolic, open('traction_admissibility_sympy.pkl', 'wb'))